# Generate hexagonal dataset

In [1]:
import os
import zipfile

import numpy as np
from nexusformat.nexus import *
import matplotlib.pyplot as plt

In [2]:
# Base directory
base_dir = "data"

xs = [15,300]

# Create the folders
for x in xs:
    folder_name = f"{x}"  # Format x to one decimal place
    full_path = os.path.join(base_dir, folder_name)
    os.makedirs(full_path, exist_ok=True)

In [3]:
for idx, T in enumerate(xs):
    # Create array
    array_size = (100, 150, 200)
    array = np.zeros(array_size)
    
    # Create coordinates
    x, y, z = np.meshgrid(np.linspace(-1.5, 1.5, array_size[0]),  # size of grid in real units
                          np.linspace(-1.5, 1.5, array_size[1]),
                          np.linspace(-1.5, 1.5, array_size[2]),
                          indexing='ij')
    
    # Grid parameters
    int_positions = [-1,0,1]  # integer crystallographic positions
    std_dev = 0.025  # in absolute coordinates (real units)
    amplitude_int = 1.0

    # Add integer-position Gaussians
    for i in int_positions:
        for j in int_positions:
            for k in int_positions:
                mean = (i, j, k)
                gaussian = amplitude_int * np.exp(-((x - mean[0]) ** 2 +
                                                    (x - mean[0]) * (y - mean[1]) +
                                                    (y - mean[1]) ** 2 +
                                                    (z - mean[2]) ** 2) / (2 * std_dev ** 2))
                array += gaussian

    # Construct NXdata with Q from -1.5 to 1.5 reciprocal lattice units
    H = NXfield(np.linspace(-1.5, 1.5, array.shape[0]), name='Qh')
    K = NXfield(np.linspace(-1.5, 1.5, array.shape[1]), name='Qk')
    L = NXfield(np.linspace(-1.5, 1.5, array.shape[2]), name='Ql')

    out_data = NXdata(NXfield(array.transpose(2, 1, 0)*1e5, name='v'))

    # Save
    nxsave(f'data/{T}/transform.nxs', out_data)
    print(f'Saved {T} K data')


Saved 15 K data
Saved 300 K data


In [6]:
for i,T in enumerate(xs):
    main_file = NXroot()
    main_file['entry'] = NXentry()
    main_file['entry']['transform'] = NXdata(NXlink(name='data',
                                               target='/entry/data/v',
                                               file=f'{T}/transform.nxs'), [L, K, H])
    main_file.entry.transform.attrs['angles'] = [90.,90.,90.]
    main_file.entry.transform.attrs['signal'] = 'data'
    main_file.entry.transform['title'] = f'{T}.000K Transform'
    print(main_file.tree)
    nxsave(f'data/hexagonal_{T}.nxs', main_file)

root:NXroot
  entry:NXentry
    transform:NXdata
      @angles = [90.0, 90.0, 90.0]
      @axes = ['Ql', 'Qk', 'Qh']
      @signal = 'data'
      Qh = float64(100)
      Qk = float64(150)
      Ql = float64(200)
      data -> 15/transform.nxs['/entry/data/v']
      title = '15.000K Transform'
root:NXroot
  entry:NXentry
    transform:NXdata
      @angles = [90.0, 90.0, 90.0]
      @axes = ['Ql', 'Qk', 'Qh']
      @signal = 'data'
      Qh = float64(100)
      Qk = float64(150)
      Ql = float64(200)
      data -> 300/transform.nxs['/entry/data/v']
      title = '300.000K Transform'


In [5]:
np.savetxt('temperatures.csv', xs, fmt='%d', delimiter=',')